0. DATASETS

In [ ]:
! gdown --id 1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
! gdown --id 1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
! gdown --id 1oiVQignVg1wvE1LXKFvOLHPl0P8GNSDp

In [ ]:
!pip install python-arango
!pip install arangopipe==0.0.70.0.0
!pip install pandas PyYAML==5.1.1 sklearn2
!pip install jsonpickle

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submit_data = pd.read_csv("submit.csv")

In [ ]:
test_data = test_data.join(submit_data["label"])

In [ ]:
# Removing null values
train_data.dropna(inplace=True)
train_data.drop(train_data.loc[train_data['text']==" "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="  "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="\n"].index, inplace=True)

# Resetting index
train_data.reset_index(inplace=True)

In [ ]:
train_data= train_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
train_data = train_data.drop(columns=["id", "index"])

In [ ]:
# Removing null values
test_data.dropna(inplace=True)
test_data.drop(test_data.loc[test_data['text']==" "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="  "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="\n"].index, inplace=True)

# Resetting index
test_data.reset_index(inplace=True)

In [ ]:
test_data= test_data.apply(lambda x: x.astype(str).str.lower())

In [ ]:
test_data = test_data.drop(columns=["id","index"])

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
! gdown --id 1pGkEyDelD7Lp7yhFML9Dt1oo-91Mqf9H
! gdown --id 1DYVzeYjjTTeHtazHfKD9FTx57h-77bkC

In [ ]:
transformed_df = pd.read_csv("pre_train.csv")
transformed_df_test = pd.read_csv("pre_test.csv")

In [ ]:
X = transformed_df
y = train_data["label"].to_frame()

In [ ]:
X_test = transformed_df_test
y_test = test_data["label"].to_frame()

In [ ]:
"""X.reset_index(inplace=True)
y.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_test.reset_index(inplace=True)

X = X.drop(columns=["index"])
y = y.drop(columns=["index"])
X_test = X_test.drop(columns=["index"])
y_test = y_test.drop(columns=["index"])"""

ArangoML

In [ ]:
y.rename(columns = {'label':'classlabel'}, inplace = True)
y_test.rename(columns = {'label':'classlabel'}, inplace = True)
Xtotal = X.join(y["classlabel"])
X_testtotal = X_test.join(y_test["classlabel"])

In [ ]:
from arangopipe.arangopipe_analytics.rf_dataset_shift_detector import RF_DatasetShiftDetector

rfd = RF_DatasetShiftDetector()
score = rfd.detect_dataset_shift(X, X_test)
print ("Detaset shift score : %2.2f" % (score))

Detaset shift score : 0.80


In [ ]:
from arangopipe.arangopipe_analytics.rf_dataset_shift_detector import RF_DatasetShiftDetector

rfd = RF_DatasetShiftDetector()
score = rfd.detect_dataset_shift(Xtotal, X_testtotal)
print ("Detaset shift score : %2.2f" % (score))

Detaset shift score : 0.80


1. distribution

In [ ]:
"""extracted_col = y["label"]
X_test.insert(3000, "classLabel", extracted_col)"""

In [ ]:
X.insert(3000, "classLabel", y["label"])
X_test.insert(3000, "classLabel", y_test["label"])

In [ ]:
X['is_train'] = 1
X_test['is_train'] = 0

In [ ]:
dataset = pd.concat([X, X_test])

In [ ]:
#drop classLabel from both
dataset = dataset.drop("classLabel", axis=1)

In [ ]:
y = dataset['is_train'].values
x = dataset.drop('is_train', axis=1).values

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
m = RandomForestClassifier()
predictions = np.zeros(y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold as SKF

skf = SKF(n_splits=5, shuffle=True, random_state=0)
for fold, (train_idx, test_idx) in enumerate(skf.split(x, y)):
 X_train, X_test = x[train_idx], x[test_idx]
 y_train, y_test = y[train_idx], y[test_idx]
 
 m.fit(X_train, y_train)
 probs = m.predict_proba(X_test)[:, 1] #calculating the probability
 predictions[test_idx] = probs

In [ ]:
from sklearn.metrics import roc_auc_score as AUC

In [ ]:
#without removing classlabels [DO NOT RERUN]
print('ROC-AUC for train and test distributions:', AUC(y, predictions))

ROC-AUC for train and test distributions: 0.7176302100050193


In [ ]:
#removing classlabels - need to drop target class label
print('ROC-AUC for train and test distributions:', AUC(y, predictions))

ROC-AUC for train and test distributions: 0.5051591897908786
